In [1]:
%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [ ]:
#How to store lidar data...
#load it
f_lidar = '../data/object/validation/velodyne/000001.bin'
points = np.fromfile(f_lidar,dtype=np.float32)
print np.shape(points)
point_per_row = points.reshape((-1,4))
print np.shape(point_per_row)
print np.shape(point_per_row.flatten())

In [ ]:
#how to store it
tmp_f = 'pointcloud.bin'
point_per_row.flatten().tofile(tmp_f)

In [2]:
#Get pointcloud from ros system
import rospy
from sensor_msgs.msg import PointCloud2, PointField
import std_msgs.msg
import sensor_msgs.point_cloud2 as pcl2
import struct

pc2_msg      = None
sub          = None
got_data     = False


def callback(msg):
    global pc2_msg
    global sub
    global got_data
    
    #for point in pcl2.read_points(msg,skip_nans=True):
        
    pc2_msg = msg
    got_data = True
    sub.unregister()
    rospy.signal_shutdown('done')
    
rospy.init_node('pc_listen',anonymous=True)
sub = rospy.Subscriber('/velodyne_points_right',PointCloud2,callback)
rospy.spin()

    


In [ ]:
print got_data
print type(pc2_msg)

In [4]:
points = []
for point in pcl2.read_points(pc2_msg,skip_nans=True):
    points.append(point)


points_np = np.array(points)

In [ ]:
plot(points_np[:,0],points_np[:,1],'.')
#we should transform the points to sensor_board_link

In [38]:

print rot.T

print ""

print np.linalg.inv(rot)

[[-1.99156132e-04 -9.77821946e-01  2.09437826e-01]
 [ 9.99999979e-01 -2.03673204e-04  0.00000000e+00]
 [ 4.26568731e-05  2.09641495e-01  9.77821966e-01]]

[[-1.99147637e-04 -9.77780237e-01  2.09632557e-01]
 [ 9.99999979e-01 -2.03664516e-04  4.05610373e-08]
 [ 4.26550535e-05  2.09428888e-01  9.77780257e-01]]


In [43]:
#From tf-echo
translation = np.array([0.030, 0.426, -0.158])
#rot = euler2mat(x=-0.000, y=-0.211, z=-1.571)
rot = get_R(0,-0.211,-1.571)get


points_sbl = np.dot(points_np[:,:3],np.linalg.inv(rot).T)-translation
plot(points_sbl[:,0],points_sbl[:,1],'.')
axis('equal')

#Fake intensity = 1
#and store as flat array
points_xyzi = np.ones([np.size(points_sbl,0),4],dtype=np.float32)
points_xyzi[:,:3] = points_sbl

#In kitty the sensor is about 0.5 higher than this, so let's remove
#-0.5 from z
points_xyzi[:,2] -= 0.5

print np.shape(points_xyzi)
#store as flat array
tmp_f = '000001.bin'
points_xyzi.flatten().tofile(tmp_f)

(21987, 4)


In [48]:
#Let's understand what the hell is going on in process_pointcloud
point_cloud = np.copy(points_xyzi)

#size z, size y, size x
voxel_size = np.array([0.4, 0.2, 0.2], dtype=np.float32)
#10 cells in up/down, 400 sideways, 352 forward
#corresponding to 4.0 m, 80.0 m sideways, 70.4 m forward
grid_size = np.array([10, 400, 352], dtype=np.int64)
lidar_coord = np.array([0, 40, 3], dtype=np.float32)
max_point_number = 35

#We add 40.0 to y so we get points 40 meters to the right at 0.0
#We add 3.0  to z so we get points 3 meters below at 0.0
shifted_coord = point_cloud[:, :3] + lidar_coord
# reverse the point cloud coordinate (X, Y, Z) -> (Z, Y, X)
# then get index by dividing with sizes
voxel_index = np.floor( shifted_coord[:, ::-1] / voxel_size).astype(np.int)

bound_x = np.logical_and(
    voxel_index[:, 2] >= 0, voxel_index[:, 2] < grid_size[2])
bound_y = np.logical_and(
    voxel_index[:, 1] >= 0, voxel_index[:, 1] < grid_size[1])
bound_z = np.logical_and(
    voxel_index[:, 0] >= 0, voxel_index[:, 0] < grid_size[0])

#Get mask for the data
bound_box = np.logical_and(np.logical_and(bound_x, bound_y), bound_z)

#Only keep points and their indices within bounds
point_cloud = point_cloud[bound_box]
voxel_index = voxel_index[bound_box]

# [K, 3] coordinate buffer as described in the paper
#The unique rows, i.e. the voxels where we have data
coordinate_buffer = np.unique(voxel_index, axis=0)

K = len(coordinate_buffer)
T = max_point_number

# [K, 1] store number of points in each voxel grid
number_buffer = np.zeros(shape=(K), dtype=np.int64)

# [K, T, 7] feature buffer as described in the paper
feature_buffer = np.zeros(shape=(K, T, 7), dtype=np.float32)

# build a reverse index for coordinate buffer
index_buffer = {}
for i in range(K):
    index_buffer[tuple(coordinate_buffer[i])] = i

for voxel, point in zip(voxel_index, point_cloud):
    index = index_buffer[tuple(voxel)]
    number = number_buffer[index]
    #add a point to the feature buffer if
    #we have less than T points in it (initially the points are zeros)
    if number < T:
        feature_buffer[index, number, :4] = point
        number_buffer[index] += 1

#Add difference from the mean to each of the T points
#as last three dimensions
feature_buffer[:, :, -3:] = feature_buffer[:, :, :3] - \
feature_buffer[:, :, :3].sum(axis=1, keepdims=True)/number_buffer.reshape(K, 1, 1)

voxel_dict = {'feature_buffer': feature_buffer,
          'coordinate_buffer': coordinate_buffer,
          'number_buffer': number_buffer}

#Okay, is this really what they said in the paper? YES


coordinate buffer =  [[  2 147  39]
 [  2 148  39]
 [  2 149  39]
 ...
 [  9 301 113]
 [  9 302 113]
 [  9 310 142]]
voxel_index:  (9385, 3) 9385
coord buff:  (1888, 3) 1888


In [42]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(points_sbl[:,0],points_sbl[:,1],points_sbl[:,2],lw=0)
ax.set_aspect('equal')

In [ ]:
points = np.fromfile(tmp_f,dtype=np.float32)
print np.shape(points)
point_per_row = points.reshape((-1,4))
print np.shape(point_per_row)

plot(point_per_row[:,0],point_per_row[:,1],'.')
axis('equal')

In [30]:
def get_R(roll,pitch,yaw):
    cos_alph = cos(yaw)
    sin_alph = sin(yaw)
    cos_beta = cos(pitch)
    sin_beta = sin(pitch)
    cos_gamma = cos(roll)
    sin_gamma = sin(roll)
    
    return np.array(
        [[cos_alph*cos_beta, 
         cos_alph*sin_beta*sin_gamma-sin_alph*cos_gamma,
         cos_alph*sin_beta*cos_gamma+sin_alph*sin_gamma],
        [sin_alph*cos_beta, 
         sin_alph*sin_beta*sin_gamma+cos_alph*cos_gamma,
        sin_alph*sin_beta*cos_gamma-cos_alph-cos_alph*sin_gamma],
        [-sin_beta,cos_beta*sin_gamma,cos_beta*cos_gamma]])

    